In [4]:
# Import necessary module
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import pymysql

In [23]:
def back_to_homepage(): # function used for return to home of redbus web page
    driver.get('https://www.redbus.in/')

def find_elements(): # get into each of Government bus list
        return driver.find_elements(By.CLASS_NAME, "rtcName")

def wait_to_webresponse(by, value): # wait to locate the element
    return WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((getattr(By, by), value))
    )
def is_element_present(by, value): # return boolean value when element available or not
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((getattr(By, by), value)))
        return True
    except:
        return False
        
def is_element_visible(by, value): # check whether element is displaying on current view point
    try:
        if is_element_present(by, value):
            element = driver.find_element(getattr(By, by), value)
            if element.is_displayed():
                return True
    except:
        return False

In [2]:
# storing the data in dictionary format to change to dataframe
data = {
    'id': [],
    'route_Departure_name': [],
    'route_Arrival_name': [],
    'route_link': [],
    'busname': [],
    'bustype': [],
    'departing_time': [],
    'duration': [],
    'reaching_time': [],
    'star_rating': [],
    'price': [],
    'seats_available': []
}

In [50]:
# Initialize the driver and ActionChains
driver = webdriver.Chrome()
actions = ActionChains(driver)

In [37]:
# go to redbus website and get government name using find_elements and text 
try:
    # Go to Google
    driver.get('http://www.google.com/')

    # Wait until the search box is present
    search_box = wait_to_webresponse('NAME', 'q')
    search_box.send_keys('redbus.in')
    search_box.submit()

    # Locate the specific link to 'redbus.in' and click it using PARTIAL_LINK_TEXT
    redbus_link = wait_to_webresponse('PARTIAL_LINK_TEXT','redbus.in')
    redbus_link.click()
    time.sleep(2)
    gov_element_name = [i.text for i in find_elements()] # get all government element name list to check and go with iterate

except Exception as e:
    # Handle the exception and display an error message
    print("Something else went wrong:", e)

In [ ]:
"""
-> This iterate will run upto 12 to get 12 governments bus information 
"""

for a in range(0,12):
    link_each_route = [] # collecting of all bus route links form all pages of gov bus element
    elements_throw_iterate = find_elements() # get all government element list to avoid the StaleElementReferenceException while iterate
    for b in range(len(elements_throw_iterate)):
        k = elements_throw_iterate[b].text
        if gov_element_name[a] == k:
            element_not_available = True
            while element_not_available:
                try:
                    actions.move_to_element(elements_throw_iterate[b]).perform()
                    elements_throw_iterate[b].click()
                    element_not_available = False
                except ElementClickInterceptedException:
                    try:
                        # Wait for the div element with id 'Carousel' to be present
                        div_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, 'div#Carousel'))
                        )
                        # Locate the span element within the div
                        span_element = div_element.find_element(By.CSS_SELECTOR, 'span.sc-ksYbfQ.nuNba').find_element(By.CSS_SELECTOR, 'span.sc-hmzhuo.dMsnin')
                        # Perform the click action on the span element
                        span_element.click()
                        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(elements_throw_iterate[j]))
                        actions.move_to_element(elements_throw_iterate[b]).perform()
                        elements_throw_iterate[b].click()
                        element_not_available = False
                    except Exception as e:
                        element_not_available = True

            #if there more than one page and it will enter into all and get all links of particular state bus detail
            if is_element_present("CLASS_NAME", "DC_117_pageTabs"):
                pagination_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
                for page in pagination_tabs:
                    for route_element in driver.find_elements(By.CLASS_NAME, "route_details"):
                        link_each_route.append(str(route_element.find_element(By.CLASS_NAME, "route").get_attribute("href")))
                    actions.move_to_element(page).perform()
                    page.click()
                    time.sleep(1)
            else:
                for route_element in driver.find_elements(By.CLASS_NAME, "route_details"):
                    link_each_route.append(str(route_element.find_element(By.CLASS_NAME, "route").get_attribute("href")))

            #get all value for using link by iterate
            for i in range(0,len(link_each_route)):
                driver.get(f"{link_each_route[i]}")
                #if there all government bus detail hide and it will unhide 
                if not (is_element_present("XPATH",'//div[@class="error-view oops-page new_oops_container"]')):
                    if is_element_present("CLASS_NAME", "button"):
                        k = driver.find_element(By.CLASS_NAME, "button")
                        actions.move_to_element(k).perform()
                        actions = ActionChains(driver)
                        k.click()
                        driver.implicitly_wait(5)
                        
                    driver.maximize_window()
                    time.sleep(1)

                    #get the element
                    element = driver.find_element(By.ID, "root")

                    #scrolling to end of particular element end
                    scrolling = True
                    window_height = 0
                    while scrolling:
                        # Scroll to the bottom of the element
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(2)  # Add a delay to allow content to load
                        
                        # Get the scroll height of the element
                        scroll_height = driver.execute_script("return arguments[0].scrollHeight;", element)
                        
                        if scroll_height == window_height:
                            scrolling = False
                        else:
                            window_height = scroll_height

                    #get element detail using 'class=bus-items' 
                    container = WebDriverWait(driver, 20).until(
                            EC.presence_of_element_located((By.CLASS_NAME, "bus-items"))
                        )
                    container_details = container.find_elements(By.XPATH,'//div[@class="clearfix row-one"]')
                    
                    # get all necessary value using container_details if get an issue while fetch then value will be null
                    for j in range(len(container_details)): 
                        try:
                            name = (container_details[j].find_elements(By.XPATH,'//div[@class="travels lh-24 f-bold d-color"]'))[j].text
                        except:
                            name = "null"
                    
                        try:
                            type = (container_details[j].find_elements(By.XPATH,'//div[@class="bus-type f-12 m-top-16 l-color evBus"]'))[j].text
                        except:
                            type = "null"
                    
                        try:
                            Departure_time = (container_details[j].find_elements(By.XPATH,'//div[@class="dp-time f-19 d-color f-bold"]'))[j].text
                        except:
                            Departure_time = "null"
                    
                        try:
                            Departure_loc = (container_details[j].find_elements(By.XPATH,'//div[@class="column-three p-right-10 w-10 fl"]/div[last()]'))[j].text
                        except:
                            Departure_loc = "null"
                    
                        try:
                            travel_total_duration = (container_details[j].find_elements(By.XPATH,'//div[@class="dur l-color lh-24"]'))[j].text
                        except:
                            travel_total_duration = "null"
                    
                        try:
                            Arrival_time = (container_details[j].find_elements(By.XPATH,'//div[@class="bp-time f-19 d-color disp-Inline"]'))[j].text
                        except:
                            Arrival_time = "null"
                    
                        try:
                            Arrival_loc = (container_details[j].find_elements(By.XPATH,'//div[@class="column-five p-right-10 w-10 fl"]/div[last()]'))[j].text
                        except:
                            Arrival_loc = "null"
                    
                        try:
                            rating = (container_details[j].find_elements(By.XPATH,'//div[@class="rating-sec lh-24"]/div/span'))[j].text
                        except:
                            rating = "null"
                    
                        try:
                            rate = (container_details[j].find_elements(By.XPATH,'//div[@class="fare d-block"]/span'))[j].text
                        except:
                            rate = "null"
                    
                        try:
                            seats_available = (container_details[j].find_elements(By.XPATH,'//div[@class="column-eight w-15 fl"]/div[1]'))[j].text
                        except:
                            seats_available = "null"

                        # append all values to respective list
                        data['id'].append(len(data['id'])+1)
                        data['route_link'].append(link_each_route[i])
                        data['busname'].append(name)
                        data['bustype'].append(type)
                        data['departing_time'].append(Departure_time)
                        data['route_Departure_name'].append(Departure_loc)
                        data['route_Arrival_name'].append(Arrival_loc)
                        data['reaching_time'].append(Arrival_time)
                        data['star_rating'].append(rating)
                        data['price'].append(rate)
                        data['duration'].append(travel_total_duration)
                        data['seats_available'].append(seats_available)
            break  # Exit the loop if the click is successful        
    time.sleep(2)
    back_to_homepage() #get back to home page
    time.sleep(1)
    wait_to_webresponse('CLASS_NAME', "rb_logo") #wait to response to go next until get element 'rb_logo'

In [58]:
# connect with MySQL database
myconnection = pymysql.connect(host='127.0.0.1',user='root',passwd='Krisrak@123')

In [ ]:
# convert data to dataframe and stored it in 'df'
df=pd.DataFrame(data)
df

In [ ]:
# create MySQL database as 'Redbus_webscraped_data'
myconnection.cursor().execute("create database Redbus_webscraped_data")

In [ ]:
#get details of columns, dtypes using for loop and join with ',' to convert for create table 
a=",".join(f"{i} {j}"
for i,j in zip(df.columns,df.dtypes)).replace("int64","int").replace("object","text")
a

In [ ]:
#create table with using variable 'a' data
myconnection.cursor().execute(f"create table Redbus_webscraped_data.bus_routes ({a})")

In [63]:
#insert all df row values using iterate and commit to save
sql = "insert into Redbus_webscraped_data.bus_routes values"
for i in range(len(df)):
    myconnection.cursor().execute(f"{sql} {tuple(df.iloc[i])}")
    myconnection.commit()